# Import Library

In [1]:
import numpy as np
import pandas as pd
import math
import random
import dill
try:
    dill.load_session('Pengujian_Normalisasi.db')
except Exception as ex:
    print("Data Belum Tersimpan!")

# Support

## Identify Label

In [2]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return np.array(distinct).astype(int), len(distinct)

## Model Evaluation Performance

### Mean + STD

In [3]:
def evaluation(arr_acc):
    mean = np.mean(arr_acc)
    varian = sum((arr_acc - mean) * (arr_acc - mean)) / len(arr_acc)
    std = math.sqrt(varian)
    return mean, std

### Pick Random Confussion Matrix

In [4]:
def toNumpyArray(coff):
    coff = np.array(coff)
    for i in range(len(coff)):
        coff[i] = np.array(coff[i])
        for j in range(len(coff[i])):
            coff[i][j] = np.array(coff[i][j])
    return coff

def pickCoff(coff):
    coff = toNumpyArray(coff)
    choosen_coff = []
    for i in range(len(coff)):
        for j in range(len(coff[i])):
            acc = 0
            for k in range(len(coff[i][j])):
                for l in range(len(coff[i][j][k])):
                    if k == l:
                        acc += coff[i][j][k][l]
            acc /= np.sum(coff[i][j])
            if acc != 1:
                choosen_coff.append(coff[i][j])
                break
            elif j == len(coff[i]) - 1:
                choosen_coff.append(coff[i][j])
    return np.array(choosen_coff)

## Euclidian Distance

In [5]:
def euclidian(X, Y, categories = []):
    euc = 0
    for i in range(len(X)):
        if not i in categories: ## similarity matrix
            euc += math.pow((X[i]-Y[i]), 2)
    euc = math.sqrt(euc)
    ##------------Similiratity Matrix-------------##
    if len(categories) != 0:
        p = len(categories)
        m = 0
        for i in range(len(categories)):
            if X[categories[i]] == Y[categories[i]]:
                m += 1
        cat_dist = (p-m)/p
        euc += cat_dist
    ##--------------------------------------------##
    return euc

# Pre Processing

## Random Data

In [6]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

## Normalisasi

In [7]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

## Missing Value

In [8]:
def handleMissValue(data):
    filled_data = np.array([[0 for i in range(len(data[0]))] for j in range(len(data))])
    cLabel = count_label(data)[1]
    arr_data = [[] for j in range(cLabel)]
    mean = []
    for i in range(len(data)):
        arr_data[int(data[i][-1])-1].append(data[i])
    counter = 0
    for i in range(len(arr_data)):
        mean.append(np.nanmean(arr_data[i], axis=0))
        for j in range(len(arr_data[i])):
            for k in range(len(arr_data[i][j])):
                if np.isnan(arr_data[i][j][k]):
                    arr_data[i][j][k] = mean[i][k]
            filled_data[counter] = arr_data[i][j]
            counter += 1
    return filled_data

## Ekstraksi File 


In [9]:
def extraction(file, is_normal = True, is_random = True):
    data = pd.read_excel(file)
    data = handleMissValue(np.array(data))
    data = pd.DataFrame(data)
    data.drop_duplicates()
    data = np.array(data)
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data.tolist())
    return data

# LMKHNCN

## Set of KNCN

### 1 NCN

In [10]:
def closest(Xx, Xx_label, Y, cLabel, nominal):
    distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
    min_distance_value = [0 for i in range(cLabel)]
    min_distance_index = [0 for i in range(cLabel)]
    for i in range(len(Xx)):
        distance[((Xx_label[i][0])-1)][i] = euclidian(Xx[i], Y, nominal)
    for i in range(cLabel):
        min_distance_value[i] = min(x for x in distance[i] if x is not None)
        min_distance_index[i] = distance[i].index(min(x for x in distance[i] if x is not None))
    return np.array(distance), min_distance_index, min_distance_value

### N NCN

In [11]:
def kncn(k, Xx, Xx_label, Y, cLabel, close_dist, nominal):
    ncn = np.array([[0 for i in range(k)] for j in range(cLabel)])
    for i in range(cLabel):
        ncn[i][0] = close_dist[i]

    for i in range (1, k):
        distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
        min_distance_value = [0 for i in range(cLabel)]
        min_distance_index = [0 for i in range(cLabel)]
        for j in range(len(Xx)):
            label = Xx_label[j][0] - 1
            check = True
            for a in range(i):
                if ncn[label][a] == j:
                    check = False
            if check:
                add = 0 + Xx[j]
                for a in range(i):
                    indeks_ncn = ncn[label][a]
                    add += Xx[indeks_ncn]
                centroid = add / (i+1)
                distance[label][j] = euclidian(Y, centroid, nominal)
        for j in range(cLabel):
            min_distance_value[j] = min(x for x in distance[j] if x is not None)
            min_distance_index[j] = distance[j].index(min(x for x in distance[j] if x is not None))
            ncn[j][i] = min_distance_index[j]
    return ncn

## Local Mean Vector

In [12]:
def localMeanVector(Xx, ncn):
    lm_ncn = [[0 for i in range(len(ncn[0]))] for j in range(len(ncn))]
    for i in range(len(ncn)):
        lm_ncn[i][0] = Xx[ncn[i][0]]
        for j in range(1, len(ncn[0])):
            add = 0 + lm_ncn[i][0]
            for a in range(j):
                add += Xx[ncn[i][a + 1]]
            lm_ncn[i][j] = add / (j+1)
    return np.array(lm_ncn)

## Harmonic Mean Distance

In [13]:
def harmonicMean(lm_ncn, Y, nominal):
    k = len(lm_ncn[0])
    hm_ncn = []
    for i in range(len(lm_ncn)):
        add = 0
        for j in range(len(lm_ncn[0])):
            distance = euclidian(Y, lm_ncn[i][j], nominal)
            ## Centroid Lokal i Tepat Pada Data Uji
            if distance == 0:
                add =+ 0
            else:
                add += (1/distance)
        ## Semua Centroid Lokal Tepat Pada Data Uji
        if add == 0:
            harmonic_mean = 0
        else:
            harmonic_mean = (k/add)
        hm_ncn.append(harmonic_mean)
    return hm_ncn

## Classification Decision

In [14]:
def decision(hm_ncn, Y_label):
    min_distance_index = hm_ncn.index(min(x for x in hm_ncn))
    return (min_distance_index + 1), Y_label[0]

# Performance Evaluation

## K Cross Fold Validation

In [15]:
def crossValidation(data, cross_val, cross_index, label_index):
    split_test = int(len(data) * (1 / cross_val))
    change_index = (cross_index - 1) * split_test
    
    for i in range(split_test):
        j = (i + change_index)
        data[i], data[j] = data[j], data[i]
    
    data_train_feature = pd.DataFrame(data[split_test:]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[split_test:]), columns=[label_index]).astype(np.int)
    data_test_feature = pd.DataFrame(data[0:split_test]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[0:split_test]), columns=[label_index]).astype(np.int)
    
    data_train_feature = np.array(data_train_feature)
    data_train_label = np.array(data_train_label)
    data_test_feature = np.array(data_test_feature)
    data_test_label = np.array(data_test_label)

    return data, data_train_feature, data_train_label, data_test_feature, data_test_label

## LMKHNCN

In [16]:
def lmkhncn(k, Xx, Xx_label, Yy, Yy_label, cLabel, nominal):
    acc = 0
    coff = np.array([[0 for i in range(jml_kelas)] for j in range(jml_kelas)])
    
    for i in range(len(Yy)):
        jarak, jarak_index_min, jarak_nilai_min = closest(Xx, Xx_label, Yy[i], cLabel, nominal)
        ncn = kncn(k, Xx, Xx_label, Yy[i], cLabel, jarak_index_min, nominal)
        local_mean = localMeanVector(Xx, ncn)
        harmonic_mean = harmonicMean(local_mean, Yy[i], nominal)
        predict, actual = decision(harmonic_mean, Yy_label[i])
        
        coff[actual-1][predict-1] += 1
        if(predict == actual):
            acc += 1
#         print(str(round((i+1)/len(Yy),2)*100)+"%. Prediksi :"+ str(predict) + " Actual :" + str(actual) + " Acc Lokal: "+ str(acc/(i+1)))
    
    acc /= len(Yy)
    return acc, coff

# Inisialisasi

In [17]:
k = 15
nominal = [8] ##Kawin Attribute
file = 'Dataset Pegawai.xlsx'
cross_val = 10
label_index = 9 #Kinerja
normalisasi = True
random_data = True
data = extraction(file, normalisasi, random_data)
arr_kelas, jml_kelas = count_label(data)

# Testing

In [ ]:
arr_coff = [[] for i in range(k)]
arr_mean_acc = [0 for i in range(k)]
arr_std_acc  = [0 for i in range(k)]

for a in range(1, k + 1): #Pengujian Parameter K
    arr_acc = [0 for i in range(cross_val)]
    for i in range(1, cross_val + 1):
        data, train_f, train_l, test_f, test_l = crossValidation(data, cross_val, i, label_index)
        acc, coff = lmkhncn(a, train_f, train_l, test_f, test_l, jml_kelas, nominal)
        arr_acc[i-1] = acc
        arr_coff[a-1].append(np.array(coff))
        print("Cross-"+str(i)+" Acc:"+ str(acc))
    mean, std = evaluation(arr_acc)
    arr_mean_acc[a-1] = mean
    arr_std_acc[a-1] = std
    print('K-{0} Mean : {1} with Standar Deviation : {2}.'.format(a,mean,std))

In [18]:
print(np.array(arr_mean_acc))
print(np.array(arr_std_acc))
# print(np.array(arr_coff))

[0.95045593 0.9656535  0.96732523 0.9674772  0.96762918 0.96717325
 0.96641337 0.9662614  0.96504559 0.96474164 0.96352584 0.96246201
 0.96170213 0.9606383  0.9600304 ]
[0.00706973 0.00391614 0.00544786 0.00456939 0.00424717 0.00379635
 0.00415925 0.00503129 0.00465949 0.00464956 0.00480589 0.00548167
 0.0059173  0.00586437 0.00604283]


## Manual Testing (Manual vs Library => Precision, Recall, dan F1-Score)

In [19]:
picked_coff = pickCoff(np.array(arr_coff))
# print(np.array(picked_coff))

### Method (Manual)

In [20]:
def calcPrecRec(coff):
    tfn = []
    tfp = []
    ttp = []
    precision = []
    recall = []
    f1score = []
    for i in range(len(coff)):
        tfn.append(np.sum(coff[i], axis = 1))
        tfp.append(np.sum(coff[i], axis = 0))
        ttp.append(np.sum(coff[i], axis = 1))
        for j in range(len(coff[i])):
            for k in range(len(coff[i][j])):
                if j == k:
                    tfn[i][j] -= coff[i][j][k]
                    tfp[i][j] -= coff[i][j][k]
                else:
                    ttp[i][j] -= coff[i][j][k]
    for i in range(len(tfn)):
        precision.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        recall.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        f1score.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        for j in range(len(tfn[i])):
            precision[i][j] = ttp[i][j]/(ttp[i][j] + tfp[i][j])
            recall[i][j] = ttp[i][j]/(ttp[i][j] + tfn[i][j])
            f1score[i][j] = 2 * precision[i][j] * recall[i][j] / (precision[i][j] + recall[i][j])
    return np.array(precision), np.array(recall), np.array(f1score)

In [21]:
def calcAvg(precision, recall, f1score):
    precision_mean = []
    recall_mean = []
    f1score_mean = []
    for i in range(len(precision)):
        precision_mean.append(np.average(precision[i]))
        recall_mean.append(np.average(recall[i]))
        f1score_mean.append(np.average(f1score[i]))
    return np.array(precision_mean), np.array(recall_mean), np.array(f1score_mean)

In [22]:
precision_arr, recall_arr, f1score_arr = calcPrecRec(picked_coff)
precision_arr_avg, recall_arr_avg, f1score_arr_avg = calcAvg(precision_arr, recall_arr, f1score_arr)

### Method (Library)

In [23]:
from sklearn.metrics import confusion_matrix #confussion matrix
from sklearn.metrics import recall_score # recall
from sklearn.metrics import precision_score #precision
from sklearn.metrics import f1_score #f1 score

In [24]:
def splitCoff(picked_coff):
    actual = []
    predict = []
    for i in range(len(picked_coff)):
        predict.append([])
        actual.append([])
        for j in range(len(picked_coff[i])):
            for k in range(len(picked_coff[i][j])):
                for l in range(picked_coff[i][j][k]):
                    if j == k:
                        actual[i].append(j)
                        predict[i].append(j)
                    else:
                        actual[i].append(j)
                        predict[i].append(k)
    return actual, predict

In [25]:
picked_coff = pickCoff(np.array(arr_coff))

precision_lib_arr = []
recall_lib_arr = []
f1score_lib_arr = []

precision_lib_arr_avg = []
recall_lib_arr_avg = []
f1score_lib_arr_avg = []

for i in range(len(picked_coff)):
    actual, predict = splitCoff(picked_coff)
    
    recall_lib = recall_score(actual[i], predict[i], average=None)
    precision_lib = precision_score(actual[i], predict[i], average=None)
    f1score_lib = f1_score(actual[i], predict[i], average=None)
    
    recall_lib_avg = recall_score(actual[i], predict[i], average='macro')
    precision_lib_avg = precision_score(actual[i], predict[i], average='macro')
    f1score_lib_avg = f1_score(actual[i], predict[i], average='macro')
    
    recall_lib_arr.append(recall_lib)
    precision_lib_arr.append(precision_lib)
    f1score_lib_arr.append(f1score_lib)
    
    precision_lib_arr_avg.append(precision_lib_avg)
    recall_lib_arr_avg.append(recall_lib_avg)
    f1score_lib_arr_avg.append(f1score_lib_avg)

### Output (Manual)

#### Precision, Recall and F1-Score (Each Class)

In [26]:
print("-------------Precision--------------")
print(precision_arr)
print()
print("---------------Recall----------------")
print(recall_arr)
print()
print("--------------F1-SCORE---------------")
print(f1score_arr)
print()

-------------Precision--------------
[[0.9330544  0.9578313  1.        ]
 [0.9459459  0.97492164 0.975     ]
 [0.9578544  0.9519231  1.        ]
 [0.9752066  0.96385545 0.97619045]
 [0.95652175 0.96460176 1.        ]
 [0.97761196 0.9537954  0.9885057 ]
 [0.9591837  0.96941894 1.        ]
 [0.97863245 0.9556213  1.        ]
 [0.963964   0.9478261  0.989011  ]
 [0.97619045 0.94207317 0.96153843]
 [0.97321427 0.94524497 1.        ]
 [0.9672131  0.9491018  0.975     ]
 [0.9839357  0.9411765  1.        ]
 [0.98712444 0.95       0.9764706 ]
 [0.96370965 0.9591837  1.        ]]

---------------Recall----------------
[[0.94491524 0.9520958  0.9886364 ]
 [0.98       0.9510703  0.962963  ]
 [0.95057034 0.96428573 0.9770115 ]
 [0.9593496  0.9756098  0.97619045]
 [0.96031743 0.9674556  0.9705882 ]
 [0.95272726 0.9763514  0.9885057 ]
 [0.96311474 0.96941894 0.9885057 ]
 [0.9423868  0.9847561  0.9885057 ]
 [0.93043476 0.97321427 0.9782609 ]
 [0.9353612  0.9716981  0.97402596]
 [0.9237288  0.98203593

#### Precision, Recall and F1-Score (Average)

In [27]:
print("-------------Precision--------------")
print(precision_arr_avg)
print()
print("---------------Recall----------------")
print(recall_arr_avg)
print()
print("--------------F1-SCORE---------------")
print(f1score_arr_avg)
print()

-------------Precision--------------
[0.9636286  0.9652892  0.9699258  0.9717509  0.97370785 0.97330433
 0.9762009  0.97808456 0.9669337  0.95993406 0.97281975 0.96377164
 0.9750374  0.9711984  0.97429776]

---------------Recall----------------
[0.9618824  0.9646778  0.9639559  0.97038335 0.9661204  0.97252816
 0.97367984 0.9718828  0.9606366  0.9603618  0.96480036 0.95879394
 0.9690259  0.9692702  0.96902657]

--------------F1-SCORE---------------
[0.9627293  0.9648214  0.9668784  0.9710335  0.9698391  0.97281885
 0.9749279  0.9747858  0.9636206  0.9599127  0.96846706 0.96118754
 0.97160023 0.9698828  0.97160625]



### Output (Library)

#### Precision, Recall and F1-Score (Each Class)

In [28]:
print("-------------Precision--------------")
print(np.array(precision_lib_arr))
print()
print("---------------Recall----------------")
print(np.array(recall_lib_arr))
print()
print("--------------F1-SCORE---------------")
print(np.array(f1score_lib_arr))
print()

-------------Precision--------------
[[0.93305439 0.95783133 1.        ]
 [0.94594595 0.97492163 0.975     ]
 [0.95785441 0.95192308 1.        ]
 [0.97520661 0.96385542 0.97619048]
 [0.95652174 0.96460177 1.        ]
 [0.97761194 0.95379538 0.98850575]
 [0.95918367 0.96941896 1.        ]
 [0.97863248 0.9556213  1.        ]
 [0.96396396 0.94782609 0.98901099]
 [0.97619048 0.94207317 0.96153846]
 [0.97321429 0.94524496 1.        ]
 [0.96721311 0.9491018  0.975     ]
 [0.98393574 0.94117647 1.        ]
 [0.98712446 0.95       0.97647059]
 [0.96370968 0.95918367 1.        ]]

---------------Recall----------------
[[0.94491525 0.95209581 0.98863636]
 [0.98       0.95107034 0.96296296]
 [0.95057034 0.96428571 0.97701149]
 [0.95934959 0.97560976 0.97619048]
 [0.96031746 0.96745562 0.97058824]
 [0.95272727 0.97635135 0.98850575]
 [0.96311475 0.96941896 0.98850575]
 [0.94238683 0.9847561  0.98850575]
 [0.93043478 0.97321429 0.97826087]
 [0.93536122 0.97169811 0.97402597]
 [0.92372881 0.98203593

#### Precision, Recall and F1-Score (Average)

In [29]:
print("-------------Precision--------------")
print(np.array(precision_lib_arr_avg))
print()
print("---------------Recall----------------")
print(np.array(recall_lib_arr_avg))
print()
print("--------------F1-SCORE---------------")
print(np.array(f1score_lib_arr_avg))
print()

-------------Precision--------------
[0.96362857 0.96528919 0.96992583 0.97175084 0.97370784 0.97330436
 0.97620088 0.97808459 0.96693368 0.95993404 0.97281975 0.96377164
 0.9750374  0.97119835 0.97429778]

---------------Recall----------------
[0.96188248 0.96467777 0.96395585 0.97038328 0.96612044 0.97252812
 0.97367982 0.97188289 0.96063665 0.96036177 0.96480037 0.95879397
 0.96902589 0.96927023 0.96902653]

--------------F1-SCORE---------------
[0.96272935 0.96482143 0.96687836 0.97103352 0.96983902 0.97281884
 0.97492794 0.97478578 0.96362055 0.9599127  0.96846703 0.9611875
 0.97160027 0.96988281 0.97160624]



# Save Data

In [ ]:
# dill.dump_session('Pengujian_Normalisasi.db')